In [17]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch

In [2]:
traindata = datasets.MNIST(
    root= 'data',
    train=True,
    transform=ToTensor(),
    download= True
)

testdata = datasets.MNIST(
    root= 'data',
    train=False,
    transform=ToTensor(),
    download= True
)

100%|██████████| 9912422/9912422 [00:00<00:00, 30539723.91it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28889981.83it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw




100%|██████████| 1648877/1648877 [00:00<00:00, 10896538.60it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4519698.40it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [26]:
traindata

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
testdata

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [10]:
print(traindata.data[1].shape)
print(traindata.data.shape)

torch.Size([28, 28])
torch.Size([60000, 28, 28])


In [9]:
testdata.data.shape

torch.Size([10000, 28, 28])

In [12]:
train_loader = DataLoader(traindata, batch_size=100, shuffle=True, num_workers=1)
test_loader = DataLoader(testdata, batch_size=100, shuffle=True, num_workers=1)


In [28]:
dataiter = iter(train_loader)
data = next(dataiter)
feat, tar = data
print(feat, tar)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

In [14]:
print(train_loader)
print(test_loader)

In [32]:
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim


class CNN(nn.Module):
    def __init__(self) :
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50,10)

    def forward(self, X):
        X = f.relu(f.max_pool2d(self.conv1(X), 2))
        X = f.relu(f.max_pool2d(self.conv2_drop(self.conv2(X)), 2))
        X = X.view(-1, 320)
        X = f.relu(self.fc1(X))
        X = f.dropout(X, training=self.training)
        X = self.fc2(X)

        return f.softmax(X)

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{(len(train_loader)*100)} ({100. * batch_idx / len(train_loader):.0f}%)]\t{loss.item(): .6f}')


def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    test_loss /= len(test_loader)
    print(f'\nTest set: Average Loss: {test_loss:.4f}, Accuracy {correct}/{(len(test_loader)*100)} ({(100. * correct / (len(test_loader)*100)):.0f}%\n)')


In [48]:
for epoch in range(1,11):
    train(epoch)
    test()

C:\Users\saira\AppData\Local\Temp\ipykernel_8728\954177959.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return f.softmax(X)


Train Epoch: 1 [0/60000 (0%)]	 2.303523
Train Epoch: 1 [2000/60000 (3%)]	 2.282403
Train Epoch: 1 [4000/60000 (7%)]	 2.150484
Train Epoch: 1 [6000/60000 (10%)]	 1.921887
Train Epoch: 1 [8000/60000 (13%)]	 1.804797
Train Epoch: 1 [10000/60000 (17%)]	 1.805525
Train Epoch: 1 [12000/60000 (20%)]	 1.740674
Train Epoch: 1 [14000/60000 (23%)]	 1.727288
Train Epoch: 1 [16000/60000 (27%)]	 1.732427
Train Epoch: 1 [18000/60000 (30%)]	 1.639903
Train Epoch: 1 [20000/60000 (33%)]	 1.663262
Train Epoch: 1 [22000/60000 (37%)]	 1.694451
Train Epoch: 1 [24000/60000 (40%)]	 1.648265
Train Epoch: 1 [26000/60000 (43%)]	 1.659808
Train Epoch: 1 [28000/60000 (47%)]	 1.634784
Train Epoch: 1 [30000/60000 (50%)]	 1.702483
Train Epoch: 1 [32000/60000 (53%)]	 1.595972
Train Epoch: 1 [34000/60000 (57%)]	 1.555587
Train Epoch: 1 [36000/60000 (60%)]	 1.677710
Train Epoch: 1 [38000/60000 (63%)]	 1.599350
Train Epoch: 1 [40000/60000 (67%)]	 1.597785
Train Epoch: 1 [42000/60000 (70%)]	 1.642197
Train Epoch: 1 [44000